In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}?",
)

prompt = template.format(country_a="Korea", country_b="Vietnam")

chat.predict(prompt)

'The distance between Korea and Vietnam is approximately 2,500 kilometers (1,550 miles) when measured in a straight line.'

In [8]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "안녕! 나는 {name}이야."),
        ("human", "What is the distance between {country_a} and {country_b}? also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language="Korean",
    name="chatbot",
    country_a="Korea",
    country_b="France",
)

chat.predict_messages(prompt)

AIMessage(content='한국과 프랑스 사이의 거리는 대략 9000km입니다. 제 이름은 AI 어시스턴트입니다. 어떻게 도와드릴까요?')